In [ ]:
analytics.attribute_search(name="city")


# Beginning of Exploratory Data Analysis

In [430]:
import warnings
warnings.filterwarnings('ignore')
from demyst.analytics import Analytics
analytics = Analytics(username="code-challenge-johnathan@demystdata.com", password="y(z%CrM32aXW^CQw8cUe2Mx83uG[")
from demyst.common import connectors
import numpy as np
import pandas as pd
df = pd.read_csv('input_file.csv', encoding = "latin", header=0)
df_copy = df.copy()
print(df_copy.shape)
analytics.validate(df_copy)


(11229, 5)


Column,Status,Description
street,All Valid,All values in this column are good to go.
state,Some Invalid,16.8% of the values of this column failed validation. One example of an invalid value is 'NJ '. Click here for documentation for this column.
city,All Valid,All values in this column are good to go.
postcode,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.


The two columns 'postcode' and 'safety_flag' aren't recognised as valid column names.
We're able to infer the valid column names from https://demyst.com/docs/demyst-live/types/.

Based on the column names found in the link listed avoce, we need to change the column name 'postcode' to 'post-code'.
'safety_flag' is also not recognised as a valid name, but we can leave it in the dataframe for now because that is our 'target' variable.
We will also need to fix the 'state' variables as there are invalid data points present as shown in the output above.

In [431]:
df_copy.rename(index=str, columns={"postcode": "post_code"}, inplace=True)
analytics.validate(df_copy)


Column,Status,Description
street,All Valid,All values in this column are good to go.
state,Some Invalid,16.8% of the values of this column failed validation. One example of an invalid value is 'NJ '. Click here for documentation for this column.
city,All Valid,All values in this column are good to go.
post_code,Some Invalid,0.1% of the values of this column failed validation. One example of an invalid value is 'nan'. Click here for documentation for this column.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.


<!-- The column that has name 'safety_flag' has been inferred to be of type 'Boolean'. This is useful to know - even though we can't use this column for a lookup search for data enrichment, the DemystData api is at least able to infer the *type* of this target variable. -->
Now that we've changed the name of 'postcode' to a valid column name (post-code), the analysis recognises this column and has shown that there is a small proportion (0.1%) of invalid data, including the presence of 'nans'.
Likewise, the 'state' variable contains invalid data, so we need to fix those invalid data points additionally.

### Clean data that has invalid inputs

There are 'nans' present in the 'state' and 'post_code' variables, so we will drop those from the dataframe

In [432]:
df_copy.isna().sum()

street         0
state          9
city           9
post_code      9
safety_flag    0
dtype: int64

In [433]:
print(df_copy.shape)
# this will drop *all* 'nans', regardless of which variable is contained
df_copy.dropna(inplace=True)
print(df_copy.shape)

(11229, 5)
(11219, 5)


We managed to remove 10 records from the input data after executing the 'dropna' function, in an 'inplace' manner

In [434]:
print(df_copy['state'].unique())

['FL' 'IN' 'VA' 'OH' 'MA' 'MN' 'WI' 'NC' 'CO' 'CA' 'NY' 'NH' 'CT' 'MD'
 'NJ' 'KY' 'WA' 'IA' 'LA' 'GA' 'IL' 'TX' 'AZ' 'DE' 'MI' 'RI' 'OR' 'AR'
 'HI' 'PA' 'SC' 'AL' 'TN' 'WY' 'NV' 'NE' 'MT' 'MS' 'OK' 'KS' 'MO' 'ID'
 'DC' 'UT' 'NM' 'ND' 'ME' 'VT' 'AK' 'SD' 'NJ  ' 'NY  ' 'CA  ' 'OR  '
 'GA  ' 'MI  ' 'AZ  ' 'TX  ' 'PA  ']


There is whitespace present in several of the data points, so we should remove them

In [435]:
# remove whitespace found in state names above
df_copy['state'] = df_copy['state'].str.strip()

There is an inconsistent schema for the 'city' variable as seen below ('VACKAVILLE' vs 'Lakeland')

In [436]:
df_copy['city'].unique()

array(['Lakeland', 'Indianapolis', 'Troutville', ..., 'VACKAVILLE',
       'San Juan Capistrano', 'White Pains'], dtype=object)

In [437]:
df_copy['city'].value_counts()

New York            245
Brooklyn            158
San Francisco       155
Los Angeles         138
Chicago              99
San Antonio          83
San Diego            83
Houston              82
San Jose             74
Bronx                65
Dallas               54
Miami                50
Atlanta              43
Philadelphia         43
Sacramento           42
Oakland              41
Austin               37
Orlando              37
Charlotte            34
Seattle              33
Phoenix              32
Jacksonville         31
El Paso              31
Las Vegas            30
Washington           27
Fort Worth           27
Long Beach           26
Portland             25
Fresno               25
Pittsburgh           25
                   ... 
Sandy Hook            1
Ambridge              1
Burney                1
Pembroke pines        1
Flanders              1
Beloit                1
king of prussia       1
North Tonawanda       1
greeley               1
fox lake              1
Gowen           


Because the majority of city names conform to the schema with a capitalised first letter for each part of the name,
we need to transform other data points (such as 'long beach' and 'VACKAVILLE') to a schema like:
'Long Beach', and 'Vackaville'

In [438]:
df_copy['city'] = df_copy['city'].str.title()


In [439]:

df_copy['city'].value_counts()

New York             261
Brooklyn             167
San Francisco        158
Los Angeles          142
Chicago              104
San Diego             88
San Antonio           85
Houston               84
San Jose              76
Bronx                 73
Miami                 54
Dallas                54
Sacramento            45
Philadelphia          43
Atlanta               43
Oakland               43
Austin                39
Orlando               37
Charlotte             36
Las Vegas             35
Seattle               35
Jacksonville          33
Phoenix               32
El Paso               32
Washington            30
Long Beach            29
Fort Worth            29
Albany                26
Pittsburgh            26
Portland              26
                    ... 
Chichester             1
Moorhead               1
Chantilly              1
Callicoon              1
Gleason                1
Lock Haven             1
Havertown              1
Minooka                1
Burien                 1


As seen above, now the 'state' variable contains data points that all conform to the same schema

Based on the 'postcode' output below, there are variable types of coding schemas present in the dataset.
We need to ensure that each postcode is of fixed length. 

In [440]:
df_copy['post_code'].unique()

array(['33812-5237', '46227', '24175-6054', ..., '14626', '92192',
       '92782'], dtype=object)

The majority of postcodes are 5 digits long, so let's ensure that *every* postcode is of that length

In [441]:
df_copy['post_code'].value_counts().head(6)

94107    25
94103    21
10001    20
10016    16
10003    14
10012    13
Name: post_code, dtype: int64

In [442]:
# we need to ensure we have a consistent schema for the postcodes
def transform_postcodes(postcode):
    '''Converts postcodes to 5-digits. Assumes that the input is a Pandas Series'''
    l = len(postcode)
    if l == 4:
        return postcode.zfill(5)
    if l == 5:
        return postcode
    else:
        return postcode[:5]


In [443]:
a = df_copy['post_code'].astype('str').apply(transform_postcodes)
a.value_counts()

94107    26
94103    22
10001    22
10016    17
10003    15
10012    13
11201    13
10019    13
11222    13
10022    12
94110    12
10013    12
10014    11
94025    11
10018    11
90069    11
92101    10
94111    10
94610    10
10010    10
92660    10
94114    10
12866    10
94109    10
11249    10
07024    10
92130     9
94105     9
10009     9
11215     9
         ..
55358     1
80751     1
11553     1
92240     1
29303     1
29172     1
51201     1
02330     1
85305     1
66772     1
93304     1
50249     1
01082     1
31058     1
37821     1
98591     1
13057     1
21046     1
15235     1
19522     1
62223     1
39038     1
78404     1
66537     1
99362     1
55396     1
32177     1
17551     1
60051     1
95124     1
Name: post_code, Length: 6597, dtype: int64

Now we're ready to apply the 'transform_postcodes' function to the 'post_code' variable

In [444]:
df_copy['post_code'] = df_copy['post_code'].astype('str').apply(transform_postcodes)


In [445]:
analytics.validate(df_copy)


Column,Status,Description
street,All Valid,All values in this column are good to go.
state,All Valid,All values in this column are good to go.
city,All Valid,All values in this column are good to go.
post_code,All Valid,All values in this column are good to go.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.


Everything appears 'good to go' for data enrichment *however*, we also need to ensure that there aren't any duplicated data points present.    


In [446]:
df_copy[df_copy.duplicated()].shape[0]

21

21 duplicates were found, so let's drop them

In [447]:
df_copy.drop_duplicates(inplace=True)


Now we're ready to search for any data sets that we could use for enrichment for our input data

In [448]:
analytics.search(df_copy)


,post_code,street,country,city,state
Option 1,☒,☒,☐,☒,☒


The above output shows that 'attom_expanded_profile_report' could be used to enrich our input dataframe.
However, we don't have the column/attribute 'country' present in our data, so we need to include that to get a proper match.
We can infer that the 'country' column can be set to 'US', as the output from running df['state'].unique() shows that each of the two letter acronyms match the 'ISO_3166-1_alpha-2' schema referenced in https://demyst.com/docs/demyst-live/types/#country
So let's add that column to the sanitised dataframe.


In [449]:
df_copy['country'] = 'US'


In [450]:
analytics.search(df_copy)


,post_code,street,country,city,state
Option 1,☒,☒,☒,☒,☒


Now all the data matches!
It's time to perform enrichment and downloading

In [503]:
analytics.validate(df_copy)

Column,Status,Description
street,All Valid,All values in this column are good to go.
state,All Valid,All values in this column are good to go.
city,All Valid,All values in this column are good to go.
post_code,All Valid,All values in this column are good to go.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
country,All Valid,All values in this column are good to go.


In [47]:
# check if enriched data already exists 
if not os.path.isfile('enriched_dataframe_attom.csv'):
    results = analytics.enrich_and_download(["attom_expanded_profile_report"], df_copy, validate=True)
    results.to_csv('enriched_dataframe_attom.csv', index=False)


Verifying providers...
Starting enrichment...
Enrich Job ID: 13191


IntProgress(value=1, max=2)

HTML(value='Checking status...')

The output of the below cell shows that we've obtained four new columns appended to our original data

In [451]:
# after reading in the enriched data from a csv, we need to drop useless columns like 'Unnamed: 0'
results = pd.read_csv('enriched_dataframe_attom.csv')
results.drop(results.filter(regex="Unname|index"),axis=1, inplace=True)
results.head(3)

,inputs.city,inputs.country,inputs.post_code,inputs.safety_flag,inputs.state,inputs.street,attom_expanded_profile_report.row_id,attom_expanded_profile_report.client_id,attom_expanded_profile_report.attom_id,attom_expanded_profile_report.error
0,Lakeland,USA,33812,False,FL,3160 Otto Dr,0,NaN,162278089.0,NaN
1,Indianapolis,USA,46227,False,IN,940 Tulip Dr,1,NaN,28461481.0,NaN
2,Troutville,USA,24175,False,VA,68 Rocky Top Rd,2,NaN,214628965.0,NaN


In [452]:
results.shape

(11198, 10)

In [169]:
from IPython.display import display
display(df_copy.head(5))
display(results.head(5))
display(df_copy.shape)
display(results.shape)

,street,state,city,post_code,safety_flag,country
0,3160 Otto Dr,FL,Lakeland,33812,False,US
1,940 Tulip Dr,IN,Indianapolis,46227,False,US
2,68 Rocky Top Rd,VA,Troutville,24175,False,US
3,3330 Taylor Blair Rd,OH,West Jefferson,43162,False,US
4,4254 Eagle Landing Parkway,FL,Orange Park,32065,False,US


,inputs.city,inputs.country,inputs.post_code,inputs.safety_flag,inputs.state,inputs.street,attom_expanded_profile_report.row_id,attom_expanded_profile_report.client_id,attom_expanded_profile_report.attom_id,attom_expanded_profile_report.error
0,Lakeland,USA,33812,False,FL,3160 Otto Dr,0,NaN,162278089.0,NaN
1,Indianapolis,USA,46227,False,IN,940 Tulip Dr,1,NaN,28461481.0,NaN
2,Troutville,USA,24175,False,VA,68 Rocky Top Rd,2,NaN,214628965.0,NaN
3,West Jefferson,USA,43162,False,OH,3330 Taylor Blair Rd,3,NaN,246515148.0,NaN
4,Orange Park,USA,32065,False,FL,4254 Eagle Landing Parkway,4,NaN,164680663.0,NaN


(11198, 6)

(11198, 10)

Let's save the results to csv for future use...

Let's do some analysis on the new data to see if there is any cleaning required.

In [453]:
results['attom_expanded_profile_report.attom_id'] = results['attom_expanded_profile_report.attom_id'].astype(str)
display(results['attom_expanded_profile_report.attom_id'].min())
display(results['attom_expanded_profile_report.attom_id'].max())
display(type(results['attom_expanded_profile_report.attom_id'].iloc[0]))

'100013224.0'

'nan'

str

Let's find the indices of the empty values

In [454]:
empty_res_idxs = results['attom_expanded_profile_report.attom_id'][results['attom_expanded_profile_report.attom_id'].isin(['nan'])].index

In [455]:
empty_res_idxs

Int64Index([   15,    25,    32,    35,    46,    69,    76,    78,    82,
               87,
            ...
            11169, 11171, 11173, 11178, 11180, 11184, 11186, 11187, 11190,
            11195],
           dtype='int64', length=2294)

Let's replace those empty atom ids with 'nans'

In [456]:
results['attom_expanded_profile_report.attom_id'].iloc[empty_res_idxs] = results['attom_expanded_profile_report.attom_id'].iloc[empty_res_idxs].replace('', np.nan)

Now we have to drop 2294 of these records with empty atom ids

In [457]:
results['attom_expanded_profile_report.attom_id'].value_counts()

nan            2294
155661423.0       4
206014472.0       3
157625010.0       3
168314141.0       2
155978117.0       2
155999620.0       2
157172850.0       2
200973617.0       2
224674708.0       2
216044453.0       2
206551040.0       2
169559254.0       2
168655763.0       2
46343898.0        2
162850659.0       2
51309196.0        2
45608305.0        2
170838616.0       2
151113878.0       2
156777902.0       2
223390422.0       2
205710214.0       2
169176504.0       2
169180531.0       2
156158432.0       2
31236220.0        2
156346735.0       2
243763327.0       2
227579808.0       2
               ... 
23260640.0        1
209460331.0       1
20154871.0        1
49454559.0        1
34455770.0        1
45945459.0        1
159161898.0       1
168052774.0       1
154396676.0       1
169548732.0       1
191090737.0       1
46698444.0        1
17158735.0        1
187561483.0       1
196178513.0       1
193253751.0       1
41618418.0        1
195815116.0       1
54427789.0        1


In [458]:
# after reading in the enriched data from a csv, we need to drop useless columns like 'Unnamed: 0'
results = pd.read_csv('enriched_dataframe_attom.csv')
results.drop(results.filter(regex="Unname|index"),axis=1, inplace=True)
display(results.shape)
results.dropna(how='all', axis=1, inplace=True) # get rid nans present of cols
results.dropna(inplace=True, axis=0) # get nans present in of rows
display(results.shape)
results.drop_duplicates(inplace=True) # drop duplicate rows
display(results.shape)
# drop row_id (not useful) and 'Unnamed :0' - this is an index 
results.drop(columns=['attom_expanded_profile_report.row_id'], inplace=True)
display(results.shape)

(11198, 10)

(8904, 8)

(8904, 8)

(8904, 7)

In [460]:
results.head(3)

,inputs.city,inputs.country,inputs.post_code,inputs.safety_flag,inputs.state,inputs.street,attom_expanded_profile_report.attom_id
0,Lakeland,USA,33812,False,FL,3160 Otto Dr,162278089.0
1,Indianapolis,USA,46227,False,IN,940 Tulip Dr,28461481.0
2,Troutville,USA,24175,False,VA,68 Rocky Top Rd,214628965.0


We need to remove the 'inputs' beginning of each column as there are no matches currently...

In [461]:
results.columns = results.columns.str.replace('inputs.', '').str.replace('attom_expanded_profile_report.', '')
results['country'] = 'US'

In [462]:
analytics.validate(results)

Column,Status,Description
city,All Valid,All values in this column are good to go.
country,All Valid,All values in this column are good to go.
post_code,Not a String Column,You must convert this column to string type.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
state,All Valid,All values in this column are good to go.
street,All Valid,All values in this column are good to go.
attom_id,Not a String Column,You must convert this column to string type.


In [465]:
results1 = results.copy()
results['post_code'] = results['post_code'].astype('str')
results['attom_id'] = results['attom_id'].astype('str')


In [466]:
# let's look at any further data sets for enrichment without using a 'strict' policy
display(analytics.search(results, strict=False))


In [467]:
analytics.product_outputs('safety_flag', all_products=True).sample(10)

,provider_name,attribute,option,type,description
1286,attom_tax_assessor,results[].fencecode,output,String,Indicates the presence and/or type of fence on...
1015,attom_recorder,results[].mortgage2interestrateminfirstchanger...,output,Number,Applies only to adjustable rate loans. The min...
1161,attom_tax_assessor,results[].taxmarketvalueland,output,Number,Market value of the land
1011,attom_recorder,results[].mortgage2documentinforideradjustable...,output,String,Flag to indicate if an adjustable rate rider w...
1054,attom_tax_assessor,results[].parcelnumberprevious,output,String,Previous parcel number assigned to the subject...
1082,attom_tax_assessor,results[].legaltractnumber,output,String,Tract number
471,attom_commercial_tax_assessor,results[].deedowner1namemiddle,output,MiddleName,The first deed owner's middle name of the prop...
1071,attom_tax_assessor,results[].propertylatitude,output,Latitude,The latitude of the property in degrees.
125,attom_commercial_pre_foreclosure,results[].publicationdate,output,Date,The date on which the client's data file was e...
809,attom_recorder,results[].transferinfomultiparcelflag,output,Number,Flag used to indicate a multiple parcel transa...


In [468]:
analytics.product_outputs('safety_flag', all_products=True).provider_name.value_counts()

attom_tax_assessor                       319
attom_commercial_tax_assessor            319
attom_recorder                           239
attom_commercial_recorder                239
attom_pre_foreclosure                    104
attom_commercial_pre_foreclosure         104
powerlytics_lfo2_zipcode_naics4_sales     46
attom_commercial_avm                      11
attom_avm                                 11
attom_expanded_profile_report              1
data-fn--818-demyst-testdf                 1
Name: provider_name, dtype: int64

As we have already enriched our initial data with the 'attom_expanded_profile_report' dataset, we can exclude that from further enrichment.
Let's make a list excluding that data set from further enrichments

In [469]:
potential_products = list(set(analytics.product_outputs('safety_flag', all_products=True).provider_name)-\
                          set(['attom_expanded_profile_report']))

In [470]:
potential_products

['attom_avm',
 'attom_commercial_recorder',
 'data-fn--818-demyst-testdf',
 'attom_tax_assessor',
 'powerlytics_lfo2_zipcode_naics4_sales',
 'attom_commercial_pre_foreclosure',
 'attom_recorder',
 'attom_commercial_avm',
 'attom_commercial_tax_assessor',
 'attom_pre_foreclosure']

In [504]:
analytics.validate(results)

Column,Status,Description
city,All Valid,All values in this column are good to go.
country,All Valid,All values in this column are good to go.
post_code,All Valid,All values in this column are good to go.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
state,All Valid,All values in this column are good to go.
street,All Valid,All values in this column are good to go.
attom_id,All Valid,All values in this column are good to go.


In [505]:
results_avm = analytics.enrich_and_download(["attom_avm"], results, validate=True)


Verifying providers...
Starting enrichment...
Enrich Job ID: 13220


IntProgress(value=1, max=2)

HTML(value='Checking status...')

In [506]:
if not os.path.isfile('results_avm_21_12.csv'):
    results_avm.to_csv('results_avm_21_12.csv', index=False)


In [508]:
df_avm = pd.read_csv('results_avm_21_12.csv')
df_avm.head(3)

,inputs.attom_id,inputs.city,inputs.country,inputs.post_code,inputs.safety_flag,inputs.state,inputs.street,attom_avm.row_id,attom_avm.client_id,attom_avm.attom_id,attom_avm.confidencescore,attom_avm.createdate,attom_avm.estimatedmaxvalue,attom_avm.estimatedminvalue,attom_avm.estimatedvalue,attom_avm.fsd,attom_avm.lastupdatedate,attom_avm.publicationdate,attom_avm.valuationdate,attom_avm.error
0,162278089,Lakeland,US,33812,False,FL,3160 Otto Dr,0,NaN,162278089.0,89.0,2019-07-03,237000.0,189000.0,213000.0,11.0,2019-07-03,2019-07-22,2019-06-28,NaN
1,28461481,Indianapolis,US,46227,False,IN,940 Tulip Dr,1,NaN,28461481.0,48.0,2019-07-03,158000.0,80000.0,119000.0,32.0,2019-07-03,2019-07-22,2019-06-28,NaN
2,214628965,Troutville,US,24175,False,VA,68 Rocky Top Rd,2,NaN,214628965.0,62.0,2019-07-03,157000.0,98000.0,127000.0,23.0,2019-07-03,2019-07-22,2019-06-28,NaN


In [ ]:
display(df_avm.shape)
display(results.shape)

In [492]:
analytics.validate(df_avm)

Column,Status,Description
attom_id,Not a String Column,You must convert this column to string type.
city,All Valid,All values in this column are good to go.
country,All Valid,All values in this column are good to go.
post_code,Not a String Column,You must convert this column to string type.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
state,All Valid,All values in this column are good to go.
street,All Valid,All values in this column are good to go.
row_id,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
client_id,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
attom_id,Not a String Column,You must convert this column to string type.


We need to clean the data into the appropriate format, as shown above.
Let's create a function to do so.

In [327]:
def clean_avm(df, target_col):
    df.columns = df.columns.str.replace('inputs.', '').str.replace(str(target_col) + '.', '')
    return df

In [545]:
df_recorder = clean_avm(results3, 'attom_recorder')
analytics.validate(results3)

Column,Status,Description
attom_id,Not a String Column,You must convert this column to string type.
city,All Valid,All values in this column are good to go.
country,All Valid,All values in this column are good to go.
post_code,All Valid,All values in this column are good to go.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
state,All Valid,All values in this column are good to go.
street,All Valid,All values in this column are good to go.
row_id,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
client_id,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
apnformatted,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.


In [288]:
def avm_preprocesser(df, target_col):
    df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    df.dropna(how='all', axis=1, inplace=True)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df.columns = df.columns.str.replace(str(target_col)+'.', '').str.replace('inputs.', '')
    return df

In [ ]:
df_avm = avm_preprocessor(df_avm, 'attom_avm')

In [491]:
df_avm = clean_avm(df_avm, 'attom_avm')

In [199]:
if not os.path.isfile('results_avm_com.csv'):
    results_avm_com = analytics.enrich_and_download(["attom_commercial_avm"], df_avm, validate=True)
    results_avm_com.to_csv('results_avm_com.csv')

Verifying providers...
Starting enrichment...
Enrich Job ID: 13203


IntProgress(value=1, max=2)

HTML(value='Checking status...')

Based on the number of records that were downloaded through both the 'attom_avm' and 'attom_commerical_avm' data sets,
this shows that the input dataset consists *only* of non-commerial/residential records.
Therefore, we can deduce that we can remove any of the commercial data sets from further enrichment pipelines.

In [200]:
results_avm.shape, results_avm_com.shape

((8904, 20), (8904, 30))

The number of records present in the different 'tax_assesor' data sets also confirm that *only* residential properties
are present in our input dataset

In [160]:
analytics.sample_data('data-fn--818-demyst-testdf',row_limit=5).columns

Provider doesn't have any sample data.


We can infer from the name that 'data-fn--818-demyst-testdf' is not going to be of interest to us.
Based on the other product descriptions, the other remaining 'attom' data sets could be used for enrichment.
The 'powerlytics_lfo2_zipcode_naics4_sales' could be of use, but the output of the cell shown below shows that there isn't an 'attom_id' column to tie in this data set to the others.

In [161]:
s = analytics.sample_data('powerlytics_lfo2_zipcode_naics4_sales', row_limit=5).columns
'attom_id' in s

False

In [201]:
updated_prods = [prod for prod in potential_products if not ('comm' in prod or 'avm' in prod or 'power' in prod \
                                                         or 'data-fn' in prod)]
updated_prods

['attom_tax_assessor', 'attom_recorder', 'attom_pre_foreclosure']

In [353]:
results.isna().sum()

city           0
country        0
post_code      0
safety_flag    0
state          0
street         0
attom_id       0
dtype: int64

In [351]:
df_avm.isna().sum()

attom_id                0
city                    0
country                 0
post_code               0
safety_flag             0
state                   0
street                  0
row_id                  0
client_id            8904
attom_id             1378
confidencescore      1378
createdate           1378
estimatedmaxvalue    1378
estimatedminvalue    1378
estimatedvalue       1378
fsd                  1378
lastupdatedate       1378
publicationdate      1378
valuationdate        1378
error                8904
dtype: int64

In [493]:
df_avm = df_avm.loc[:,~df_avm.columns.duplicated()]
df_avm['attom_id'] = df_avm['attom_id'].astype(str)
df_avm['post_code'] = df_avm['post_code'].astype(str)


In [486]:
df_avm.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df_avm.dropna(how='all', axis=1, inplace=True)
df_avm.dropna(inplace=True)
df_avm.drop_duplicates(inplace=True)
df_avm = df_avm.loc[:,~df_avm.columns.duplicated()]


In [487]:
df_avm['attom_id'].head(2)

0    162278089.0
1     28461481.0
Name: attom_id, dtype: object

In [501]:
results['attom_id'] = results['attom_id'].astype(float)
results['attom_id'] = results['attom_id'].astype(int)
results['attom_id'] = results['attom_id'].astype(str)


In [502]:
analytics.validate(results)

Column,Status,Description
city,All Valid,All values in this column are good to go.
country,All Valid,All values in this column are good to go.
post_code,All Valid,All values in this column are good to go.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
state,All Valid,All values in this column are good to go.
street,All Valid,All values in this column are good to go.
attom_id,All Valid,All values in this column are good to go.


In [495]:
df_avm.shape

(8904, 19)

In [544]:
updated_prods

['attom_tax_assessor', 'attom_recorder', 'attom_pre_foreclosure']

In [546]:
analytics.validate(results)

Column,Status,Description
city,All Valid,All values in this column are good to go.
country,All Valid,All values in this column are good to go.
post_code,All Valid,All values in this column are good to go.
safety_flag,Unrecognized Column Name,This column name is not supported. Click here for a list of all supported column names.
state,All Valid,All values in this column are good to go.
street,All Valid,All values in this column are good to go.
attom_id,All Valid,All values in this column are good to go.


In [547]:
updated_prods = ['attom_tax_assessor',
             'attom_pre_foreclosure',
             'attom_avm'] 

prod_dict_3 = {}
for idx, prod in enumerate(updated_prods):
    if prod_dict_3.get(str(idx)) is None and not os.path.isfile(f'{prod}_{idx}_22_11.csv'):
        prod_dict_3[str(idx)] = analytics.enrich_and_download([prod], results, validate=True)
        if not os.path.isfile(f'{prod}_{idx}_22_11.csv'):
            prod_dict_2[str(idx)].to_csv(f'{prod}_{idx}_22_11.csv', index=False)
    else:
        prod_dict_3[str(idx)] = pd.read_csv(f'{prod}_{idx}_22_11.csv')
full_df = pd.concat([df_recorder, *prod_dict_3.values()],axis=1)


Verifying providers...
Starting enrichment...
Enrich Job ID: 13222


IntProgress(value=1, max=2)

HTML(value='Checking status...')

Verifying providers...
Starting enrichment...
Enrich Job ID: 13223


IntProgress(value=1, max=2)

HTML(value='Checking status...')

Verifying providers...
Starting enrichment...
Enrich Job ID: 13225


IntProgress(value=1, max=2)

HTML(value='Checking status...')

In [395]:
cleaned_full_df = avm_preprocesser(full_df)
cleaned_full_df.isnull().sum()

avm_preprocesser() missing 1 required positional argument: 'target_col'


In [571]:
full_df = pd.concat([df_recorder, *prod_dict_3.values()],axis=1)
print(full_df.shape)
full_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
full_df.dropna(how='all', axis=1, inplace=True)
full_df.drop_duplicates(inplace=True)
full_df.columns = full_df.columns.str.replace('inputs.', '').str.replace(str('attom_avm')+'.', '')
print(full_df.shape)

(8904, 709)
(8904, 653)


In [575]:
full_df.isna().sum().sum()

3943721

In [576]:
full_df.dtypes.value_counts()

object     569
float64     69
int64       11
bool         4
dtype: int64

In [594]:
new_cols = []
for col in full_df.columns.tolist():
    if len(col.split('.'))>=2:
        print(col.split('.')[0])
        new_cols.append(col.split('.')[1])
    else:
        new_cols.append(col)
full_df.columns = new_cols
full_df.sample(1).T

attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_assessor
attom_tax_as

,6723
attom_id,146609930
city,Homestead
country,US
post_code,33035
safety_flag,False
state,FL
street,1612 S Goldeneye Ln
row_id,6723
apnformatted,NaN
apnoriginal,NaN


In [596]:
full_df.isna().sum().sum()

3943721

In [652]:
cleaned_res = full_df.copy()

cleaned_res.dropna(how='all', axis=1, inplace=True)
cleaned_res.drop_duplicates(inplace=True)
for x in cleaned_res:
    if 'date' in x:
        cleaned_res[x] = pd.to_datetime(cleaned_res[x]).dt.year
        cleaned_res = cleaned_res.loc[:,~cleaned_res.columns.duplicated()] 

cleaned_res.shape

(8904, 565)

In [653]:
cleaned_res.isna().sum().sum()

3552425

In [654]:
results_df_clean = cleaned_res.dropna(how='all', axis=1) # drop empty columns (must perform this BEFORE dropping Nan rows)

results_df_clean['estimatedminvalue'].sample(10)

4826    684000.0
6409    208000.0
4748    151000.0
4105    519000.0
4       380000.0
2245     99000.0
4955    137000.0
449     135000.0
7096    232000.0
8597         NaN
Name: estimatedminvalue, dtype: object

In [655]:
# drop records with large 'NA' percentage
# let's set that threshold arbitrarily to 80%

no_records = results_df_clean.shape[0]
bad_cols = []
for col in results_df_clean.columns:
    if results_df_clean[col].isna().sum()/no_records >= 0.8:
        bad_cols.append(col)
results_df_clean.drop(bad_cols, inplace=True, axis=1)
results_df_clean.shape

(8904, 229)

In [656]:
results_df_clean.dtypes.value_counts()

object     191
float64     35
int64        2
bool         1
dtype: int64

In [648]:
results_df_clean.select_dtypes(include='float').head().T


,0,1,2,3,4
armslengthflag,0.0,0.0,0.0,0.0,0.0
documentrecordingcountyfips,12105.0,18097.0,51023.0,39097.0,12019.0
granteemailaddresshousenumber,3160.0,940.0,NaN,3330.0,4254.0
granteemailaddresszip,33812.0,46227.0,NaN,43162.0,32065.0
granteemailaddresszip4,5237.0,2321.0,NaN,NaN,2631.0
instrumentdate,2009.0,2018.0,NaN,2015.0,2018.0
lastupdated,2014.0,2018.0,2016.0,2015.0,2019.0
mortgage1amount,NaN,20950.0,10000.0,212000.0,107000.0
mortgage1interestrate,NaN,0.0,NaN,0.0,0.0
mortgage1interestratetype,NaN,1.0,1.0,0.0,0.0


### It is clear that we're going to have to use some kind of imputation strategy to handle the presence of 'Nan's 
### in certain columns  - there is some kind of trade-off between keeping these NaNs with a 'mean' imputation strategy versus ditching them completely. Let's find out.
### A good starting place would be to use the library 'sklearn_pandas', as well as the 'sklearn' imputers available

In [330]:
df_avm_2 = df_avm.copy()
cleaned_product_dict = {}
product_name_dict = {}
raw_data_dixt = {}
for idx, (name, dataset) in enumerate(prod_dict.items()):
    col_names = dataset.columns
    product_name_dict[str(idx)] = list(set([str(name).split('.')[0] for name in col_names if 'inputs' not in name]))
    cur_name = product_name_dict[str(idx)].pop()
    cleaned_product_dict[name+str(idx)] = avm_preprocesser(dataset, cur_name)
    df_avm_2 = pd.concat([df_avm_2, cleaned_product_dict[name+str(idx)]], axis=1)
    df_avm_3 = pd.concat([df_avm, dataset], axis=1)
assert df_avm_2.shape[1] == prod_dict['0'].shape[1] + prod_dict['1'].shape[1] + prod_dict['2'].shape[1] + df_avm.shape[1]

In [ ]:
# import pandas_profiling as pf
# df_avm.profile_report(style={'full_width':True})


In [651]:
stats = analytics.report(df_avm)
print(stats.head(5))


  product_name  product_match_rate   attribute_name  attribute_fill_rate  \
0    attom_avm               100.0         attom_id                84.52   
1    attom_avm               100.0        client_id                 0.00   
2    attom_avm               100.0  confidencescore                84.52   
3    attom_avm               100.0       createdate                84.52   
4    attom_avm               100.0            error                 0.00   

  attribute_type  unique_values  \
0        float64           7514   
1        float64              0   
2        float64             80   
3         object             14   
4        float64              0   

                                  most_common_values  cardinality  \
0  {"31236220.0": 2, "322428212.0": 2, "20512479....        99.84   
1                                                 {}         0.00   
2  {"87.0": 228, "83.0": 223, "85.0": 207, "86.0"...         1.06   
3  {"2019-07-03": 7419, "2019-05-08": 24, "2018-1...    

In [650]:
whos DataFrame

Variable           Type         Data/Info
-----------------------------------------
dataset            DataFrame    Empty DataFrame\nColumns:<...>n\n[0 rows x 116 columns]
df                 DataFrame                             <...>n[11229 rows x 5 columns]
df_avm             DataFrame          inputs.attom_id    <...>n[8904 rows x 20 columns]
df_avm_2           DataFrame             attom_id        <...>[8904 rows x 682 columns]
df_avm_3           DataFrame             attom_id        <...>[8904 rows x 136 columns]
df_copy            DataFrame                             <...>n[11198 rows x 6 columns]
df_recorder        DataFrame           attom_id          <...>[8904 rows x 248 columns]
full_df            DataFrame           attom_id          <...>[8904 rows x 653 columns]
input_f            DataFrame                             <...>field  94533-6834   False
q                  DataFrame           inputs.attom_id   <...>[11198 rows x 20 columns]
results            DataFrame        

In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from xgboost.sklearn import XGBRegressor
b = pd.read_csv('data/updated_fixed_data.csv')
b['name'] = b['name'].str.lower()
b = b[~(b['name'].str.contains('portable')|b['name'].str.contains('hannah')|b['name'].str.contains('ruoyi'))]
b.drop(['name', 'children_age', 'id', 'resp_b', 'support_name', 'support_result', 'Unnamed: 0'], axis=1, inplace=True)
col_list = b.columns
new_col_list = list(col_list[:9]) + list(col_list[10:])
new_col_list.extend(['total_a'])
b = b[new_col_list]
b = handle_duplicates(b, target_index=-1, strategy='ignore', ignore_cols=[])

categorical_cols_ordinal = ['health_a', 'health_b']
categorical_cols_normal = ['child_support_new_encoding', 'no_children_flag']
continuous_cols = set(b.columns) - set(categorical_cols_ordinal) - set(categorical_cols_normal) - set(['total_a'])
estimator = XGBRegressor()
estimator = LinearRegression()
mapper = DataFrameMapper(
  [([continuous_col], StandardScaler()) for continuous_col in continuous_cols] +
  [(categorical_col, LabelBinarizer()) for categorical_col in categorical_cols_normal] +
  [([categorical_col], OrdinalEncoder()) for categorical_col in categorical_cols_ordinal],
  input_df=True, df_out=True
)

pipeline = Pipeline(
  [("mapper", mapper),
  ("estimator", estimator)]
)

X_train, X_test, Y_train, Y_test = train_test_split(b[b.columns.difference(['total_a'])], b['total_a'])
pipeline.fit(X_train, Y_train)
print(mean_absolute_error(pipeline.predict(X_test), Y_test))
#pipeline.fit(b.copy(), b["total_a"])

#pipeline.predict(b)